In [ ]:
# 5-5-strength-duration

In [ ]:


class StrengthDuration(object):
    """
    For a range of durations, this experiment checks the stimulus size needed
    to excite a cell.

    Accepts the following input arguments:

    ``model``
        The model to run the experiment with.
    ``ivar``
        A variable that can be used as the stimulus current.  It's value will
        be set as ``pace * amplitude`` where pace is a variable bound to the
        pacing signal and amplitude is varied by the method.
    ``vvar``
        A variable that indicates the membrane potential. If not specified (or
        given as ``None``), the method will search for a variable labeled as
        ``membrane_potential``.

    """
    def __init__(self, model, ivar, vvar=None):
        # Clone model
        self._model = model.clone()
        del model

        # Get stimulus current variable
        if isinstance(ivar, myokit.Variable):
            ivar = ivar.qname()
        self._ivar = self._model.get(ivar)
        del ivar

        # Get membrane potential variable
        if vvar is None:
            self._vvar = self._model.label('membrane_potential')
            if self._vvar is None:
                raise ValueError(
                    'This method requires the membrane potential variable to'
                    ' be passed in as `vvar` or indicated in the model using'
                    ' the label `membrane_potential`.')
        else:
            if isinstance(vvar, myokit.Variable):
                vvar = vvar.qname()
            self._vvar = self._model.get(vvar)
        del vvar

        # Get time variable
        self._tvar = self._model.time()

        # Unbind any existing pace variable
        var = self._model.binding('pace')
        if var is not None:
            var.set_binding(None)

        # Create new pacing variable
        c = self._ivar.parent(myokit.Component)

        def add_variable(c, name):
            try:
                return c.add_variable(name)
            except myokit.DuplicateName:
                i = 2
                n = name + str(i)
                while True:
                    try:
                        return c.add_variable(n)
                    except myokit.DuplicateName:
                        i += 1
                        n = name + str(i)
        self._pvar = add_variable(c, 'pace')
        self._pvar.set_binding('pace')
        self._pvar.set_rhs(0)

        # Create new amplitude variable
        self._avar = add_variable(c, 'amplitude')
        self._avar.set_rhs(0)

        # Set rhs of current variable
        if self._ivar.is_state():
            self._ivar.demote()
        self._ivar.set_rhs(myokit.Multiply(self._pvar.lhs(), self._avar.lhs()))

        # Set default parameters
        self.set_currents()
        self.set_precision()
        self.set_threshold()
        self.set_times()

        # No data yet!
        self._data = None

    def run(self, debug=False):
        """
        Runs the experiment, returning a :class:`myokit.DataLog` with the
        entries ``duration`` and ``strength``, where each strenght is the
        minimum required to create a depolarisation at the corresponding
        duration.
        """
        if self._data is None:
            self._run(debug)
        return self._data

    def _run(self, debug=False):
        """
        Inner version of run()
        """
        if debug:
            import traceback

        # Create simulation
        s = myokit.Simulation(self._model)

        # Variables to log
        vvar = self._vvar.qname()

        # Output data
        durations = np.array(self._durations, copy=True)
        amplitudes = np.zeros(durations.shape)

        # Test every duration
        for k, duration in enumerate(durations):
            if debug:
                print('Testing duration: ' + str(duration))
            s.set_protocol(myokit.pacing.blocktrain(self._time + 1, duration))
            a1 = self._amin
            a2 = self._amax

            # Test minimum amplitude
            s.reset()
            s.set_constant(self._avar, a1)
            try:
                d = s.run(self._time, log=[vvar]).npview()
                t1 = (np.max(d[vvar]) > self._threshold)
            except Exception:
                if debug:
                    traceback.print_exc()
                t1 = False
            if debug:
                print(t1)

            # Test maximum amplitude
            s.reset()
            s.set_constant(self._avar, a2)
            try:
                d = s.run(self._time, log=[vvar]).npview()
                t2 = (np.max(d[vvar]) > self._threshold)
            except Exception:
                if debug:
                    traceback.print_exc()
                t2 = False
            if debug:
                print(t2)
            if t1 == t2:
                # No zero crossing found
                amplitudes[k] = np.nan
                if debug:
                    print('> no zero crossing')
                continue

            # Zero must lie in between. Start bisection search
            a = 0.5 * a1 + 0.5 * a2
            for j in range(0, self._precision):
                s.reset()
                s.set_constant(self._avar, a)
                try:
                    d = s.run(self._time, log=[vvar]).npview()
                except Exception:
                    if debug:
                        traceback.print_exc()
                    break
                t = (np.max(d[vvar]) > self._threshold)
                if t1 == t:
                    a1 = a
                else:
                    a2 = a
                a = 0.5 * a1 + 0.5 * a2
            amplitudes[k] = a
            if debug:
                print('> ' + str(a))

        # Set output data
        self._data = myokit.DataLog()
        self._data['duration'] = durations
        self._data['strength'] = amplitudes

    def set_currents(self, imin=-250, imax=0):
        """
        Sets the range of current levels tested.

        ``imin``
            The lowest (or most negative) current amplitude to test.
        ``imax``
            The greatest (or least negative) current amplitude to test.
        """
        amin = float(imin)
        amax = float(imax)
        if amin >= amax:
            raise ValueError(
                'Minimum current amplitude must be smaller than maximum'
                ' value.')
        self._amin = amin
        self._amax = amax
        self._data = None

    def set_precision(self, precision=10):
        """
        Sets the number of different amplitudes tried. This is done using a
        bisection algorithm, so low values of ``precision`` can still produce a
        good result.
        """
        precision = int(precision)
        if precision < 1:
            raise ValueError('The number of tries must be greater than zero.')
        self._precision = precision
        self._data = None

    def set_threshold(self, threshold=10):
        """
        Sets the level above which the membrane potential must rise to count as
        a depolarization.
        """
        self._threshold = float(threshold)
        self._data = None

    def set_times(self, tmin=0.2, tmax=2.0, dt=0.1, twait=50):
        """
        Sets the tested stimulus durations.

        ``tmin``
            The smallest stimulus time tested.
        ``tmax``
            The largest stimulus time tested.
        ``dt``
            The step size when going from tmin to tmax.
        ``twait``
            The duration of the experiment that tries to measure a
            depolarization.

        """
        tmin = float(tmin)
        tmax = float(tmax)
        if tmin < 0:
            raise ValueError('Minimum time cannot be negative.')
        if tmax < 0:
            raise ValueError('Maximum time cannot be negative.')
        if tmin >= tmax:
            raise ValueError(
                'The maximum time must be greater than the minimum time.')
        dt = float(dt)
        if dt <= 0:
            raise ValueError('The step size must be greater than zero.')
        twait = float(twait)
        if twait < 0:
            raise ValueError('The time "twait" must be greater than zero.')
        self._durations = np.arange(tmin, tmax, dt)
        self._time = twait
        self._data = None